In [1]:
!pip install 'crewai[tools]'
# [tools] 옵션은 CrewAI에서 제공하는 추가적인 도구(plugins) 및 확장 기능을 설치합니다

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of lancedb to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.9/514.9 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.4 MB/s eta 0:00:00
  

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력
os.environ["SERPER_API_KEY"] = "" #serper 키 입력

In [3]:
from crewai import Agent
from crewai_tools import SerperDevTool
search_tool = SerperDevTool() # 구글 검색 API를 활용해 정보를 검색하는 기능을 제공

searching = Agent(
  role='Searching',
  goal='{topic}에 대한 적절한 내용을 찾아주세요',
  verbose=True,
  memory=True,
  backstory=(
    "당신은 주제에 대한 검색에 최적화되어 있습니다."
  ),
  tools=[search_tool],
  allow_delegation=True # 다른 에이전트에게 작업을 위임할 수 있도록 허용
) # searching 에이전트

answer = Agent(
  role='Writing',
  goal='{topic}에 대한 내용을 설명하세요',
  verbose=True,
  memory=True,
  backstory=(
    "간결하면서도 명확하게 작성하세요."
  ),
  tools=[search_tool], # answer도 검색 도구를 사용할 수 있음
  allow_delegation=False
) # answer 에이전트

In [5]:
from crewai import Task # CrewAI에서 특정 작업을 정의하고 실행하는 데 사용되는 클래스

search_task = Task(
  description=(
    "{topic}에서 주목할 만한 키워드를 파악하세요. 장점과 단점을 식별하고 전반적인 관점으로 분석하세요."
  ),
  expected_output='{topic}에 검색 결과를 3문단 분량으로 정리하세요.',
  tools=[search_tool],
  agent=searching,
) # 첫번째 작업, searching 에이전트를 사용하여 {topic}에 대한 검색 결과를 작성

answer_task = Task(
  description=(
    "{topic}에 대한 통찰력 있는 기사를 작성하세요. 최신 동향과 그것의 파급력을 중점적으로 다루세요."
  ),
  expected_output='{topic}에 관한 4문단짜리 기사로, 마크다운 형식으로 작성하세요.',
  tools=[search_tool],
  agent=answer,
  async_execution=False,
  output_file='/content/new-blog-post.md' # 결과를 /content/new-blog-post.md 파일에 저장
) # 두번째 작업, answer 에이전트를 사용하여 {topic}에 대한 기사를 작성

In [6]:
from crewai import Crew, Process
crew = Crew( # Crew 객체 설정
  agents=[searching, answer], # Crew 객체에 포함된 에이전트 리스트
  tasks=[search_task, answer_task], # Crew 객체에 포함된 작업 리스트
  process=Process.sequential, # 작업을 순차적으로 실행
  memory=True, # 대화 기록을 메모리에 저장하여 재사용
  cache=True,
  max_rpm=100,
  share_crew=True
)

In [7]:
result = crew.kickoff(inputs={'topic': 'AI 에이전트가 AI 산업에 미치는 영향은?'}) # Crew 객체에 정의된 작업과 에이전트를 실행
print(result)

# Agent: Searching
## Task: AI 에이전트가 AI 산업에 미치는 영향은?에서 주목할 만한 키워드를 파악하세요. 장점과 단점을 식별하고 전반적인 관점으로 분석하세요.


# Agent: Searching
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"AI agents impact on the AI industry advantages disadvantages analysis\"}"
## Tool Output: 
{'searchParameters': {'q': 'AI agents impact on the AI industry advantages disadvantages analysis', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "What are the risks and benefits of 'AI agents'? | World Economic Forum", 'link': 'https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/', 'snippet': 'In a world of talent scarcity, AI agents can help to close skills gaps in various industries, where human expertise is lacking or in high demand ...', 'position': 1}, {'title': 'Top 5 Advantages and Disadvantages of Using AI Agents', 'link': 'https://hardiks.medium.com/top-5-advantages-and-disadvantages-of-using-ai-agents-31b9213e705a', 'snippet': 'AI agents 